# Using a Bootstrap Method to Choose the Sample Fraction in Tail Index Estimation
### Generation of data

#### Distribution
On peut utiliser des données i.i.d. :
- [x] Student-t
- [x] type 2 (=Frechet) extreme value distribution
- [ ] generalized extreme value distribution (pas dans l'article)

On peut utiliser des données dépendantes :
- [ ] MA(1) process avec des variables cachées de Student-t
- [ ] Stochastic processes

On peut utiliser des données réelles
- [ ] Données financières

#### Nombre d'échantillons
2_000 à 20_000 échantillons


In [23]:
import numpy as np
from scipy.stats import t, invweibull, genextreme
import matplotlib.pyplot as plt


### Simulation variables de Student

# df = 10
# r = t.rvs(df, size=10000)
# plt.hist(r, density=True, bins=100, log=False)

# fig, ax = plt.subplots(1, 1)
# mean, var, skew, kurt = t.stats(df, moments='mvsk')
# x = np.linspace(t.ppf(0.01, df),
#                 t.ppf(0.99, df), 100)
# ax.plot(x, t.pdf(x, df),
#        'r-', lw=5, alpha=0.6, label='t pdf')

# rv = t(df)
# ax.plot(x, rv.pdf(x), 'k-', lw=2, label='frozen pdf')

# vals = t.ppf([0.001, 0.5, 0.999], df)
# np.allclose([0.001, 0.5, 0.999], t.cdf(vals, df))

# ax.hist(r, density=True, histtype='stepfilled', alpha=0.2)
# ax.legend(loc='best', frameon=False)
# plt.show()

def get_student_sample(degrees_freedom, size_sample, get_plot=False, log_plot=False, nb_bins=100):
    # Le nombre de degrés de liberté correspond au nombre de paramètres
    # Si degrees_freedom < 1 : pas d'espérance
    r = t.rvs(degrees_freedom, size=size_sample)
    if get_plot:
        plt.hist(r, density=True, bins=nb_bins, log=log_plot, label="student_samples", alpha = 0.8)
        plt.legend()
    return r


### Simulation type II EVD (Frechet distribution)


def get_frechet_sample(inv_gamma, size_sample, get_plot=False, log_plot=False, nb_bins=100):
    r = invweibull.rvs(inv_gamma, size=size_sample)
    if get_plot:
        plt.hist(r, density=True, bins=nb_bins, log=log_plot, label="frechet_samples", alpha = 0.6)
        plt.legend()
    return r


# # Trouver la relation entre X suit frechet et Y suit GEV
# def get_frechet_sample_with_genextreme(inv_gamma, size_sample, get_plot=False, log_plot=False, nb_bins=100):
#     # inv_gamma correspond au shape param
#     # Si inv_gamma < 1 pas d'esp
#     c = -1/inv_gamma
#     sigma = 1
#     mu = 0
#     r = genextreme.rvs(c, size=size_sample)
# #     r = (1+1/c*r)**(-c)  # TODO : Trouver la relation entre X suit frechet et Y suit GEV
#     if get_plot:
#         plt.hist(r, density=True, bins=nb_bins, log=log_plot, label="frechet_samples_extreme", alpha = 0.6)
#         plt.legend()
#     return r
    

### Estimation of optimal tail index with de_haan_1998

#### On définit les estimateurs de gamma
- [x] estimateur ordre 1
- [x] estimateur ordre 2

#### On applique la méthode de de_haan_1998
- [x] estimateur asymptotique avec du bootstrap
- [ ] technique pour optimiser le bootstrap
- [ ] déduction de la taille de bootstrap optimale ? ou bien simplement direct intervalle de confiance en gamma ?


In [57]:
### On définit les estimateurs de gamma de de Haan 1998
### On définit l'estimateur de la variance asymptotique
### on minimise cet estimateur par rapport à k pour une taille de bootstrap donnée

from random import choices
from tqdm.notebook import tqdm


def gamma_moment_1_estimator(k, sample):
    sorted_sample = np.sort(sample)
    k_largest_samples = sorted_sample[-k:]
    kest_largest = sorted_sample[-k]
    moment_1_estimator = np.mean(np.log(k_largest_samples)-np.log(kest_largest))
    return moment_1_estimator

# # Test du moment ordre 1
# inv_gamma = 5
# size_sample = 10000
# r1 = get_frechet_sample(inv_gamma, size_sample)
# gamma_moment_1_estimator(1000,r1)

def gamma_moment_2_estimator(k, sample):
    sorted_sample = np.sort(sample)
    k_largest_samples = sorted_sample[-k:]
    kest_largest = sorted_sample[-k]
    moment_2_estimator = np.mean((np.log(k_largest_samples)-np.log(kest_largest))**2)
    return moment_2_estimator/(2*gamma_moment_1_estimator(k, sample))

# # Test du moment ordre 2
# inv_gamma = 3
# size_sample = 10000
# r1 = get_frechet_sample(inv_gamma, size_sample)
# gamma_moment_2_estimator(1000,r1)


def Q_bootstrapped_quadratic_asymp_error_estimator(k1, n1_bootstrap, sample, monte_carlo_steps):
    # On calcule Q par Monte-Carlo
    q_values_list = list()
    for i in range(monte_carlo_steps):
        bootstrapped_sample = choices(sample, k=n1_bootstrap)
        gamma_moment_1 = gamma_moment_1_estimator(k1, bootstrapped_sample)
        gamma_moment_2 = gamma_moment_2_estimator(k1, bootstrapped_sample)
        q_value = (gamma_moment_2*(2*gamma_moment_1) - 2*gamma_moment_1**2)**2
        q_values_list.append(q_value)
    return np.mean(q_values_list)


def find_argmin_Q(n1_bootstrap, sample,
                  k_min=5, k_max="default", step="default",
                  monte_carlo_steps=2000,
                  plot_q_currents = False):
    if k_max == "default":
        k_max = int(n1_bootstrap/4*3)
    if step == "default":
        step = int(n1_bootstrap//20)
    list_of_k = range(k_min, k_max, step)
    argmin_k = k_min
    min_q = Q_bootstrapped_quadratic_asymp_error_estimator(k_min, n1_bootstrap, sample, monte_carlo_steps)
    list_q_current = list()
    for k_current in tqdm(list_of_k, desc = "q_minimzation"):
        q_current = Q_bootstrapped_quadratic_asymp_error_estimator(k_current, n1_bootstrap, sample, monte_carlo_steps)
        list_q_current.append(q_current)
        if q_current < min_q:
            min_q = q_current
            argmin_k = k_current
    if plot_q_currents:
        opti_gamma = gamma_moment_1_estimator(argmin_k, sample)
        # TODO add description k_min..etc...
        plt.plot(list_of_k, list_q_current, label="q values")
        plt.title(f"minimization of Q, k_min={argmin_k}; gamma value is {opti_gamma}")
        plt.legend()
        plt.show()
    return argmin_k

# # Test find_argmin_Q
# inv_gamma = 3
# size_sample = 10000
# r1 = get_frechet_sample(inv_gamma, size_sample)
# find_argmin_Q(500, r1, plot_q_currents=True)


def test_plot_q_minimization(n1_bootstrap, sample, nb_minimizations):
    list_argmin_k = list()
    for i in tqdm(range(nb_minimizations), desc = "iteration minimization"):
        argmin_k = find_argmin_Q(n1_bootstrap, sample, plot_q_currents=True)
        list_argmin_k.append(argmin_k)
    plt.hist(list_argmin_k)
    plt.title("Histogram of k minimization")
    plt.show()
    return list_argmin_k

# Test plot_q_minimization
inv_gamma = 3
size_sample = 10000
r1 = get_frechet_sample(inv_gamma, size_sample)
test_plot_q_minimization(500, r1, 100)

In [ ]:
### On applique l'algo de de Haan_1998


def de_haan_1998(n1_bootstrap, sample):
    argmin_k1 = find_argmin_Q(n1_bootstrap, sample,
                              k_min=5)
    n2_bootstrap = int((n1_bootstrap/len(sample))**2*len(sample))
    argmin_k2 = find_argmin_Q(n2_bootstrap, sample,
                              k_min=5)
    exposant = (np.log(n1_bootstrap) - np.log(argmin_k1))/np.log(n1_bootstrap)
    factor_2 = (np.log(argmin_k1)**2/(2*np.log(n1_bootstrap) - np.log(argmin_k1))**2) ** exposant
    k0_opti = argmin_k1**2/argmin_k2 * factor_2
    return k0_opti


# Test de_haan_opti
list_gamma = list()
for i in tqdm(range(100), desc = "de_haan_histogram"):
    inv_gamma = 3
    size_sample = 20000
    r1 = get_frechet_sample(inv_gamma, size_sample)
    n1_bootstrap = int(size_sample**(4/5))
    k0_opti = int(de_haan_1998(n1_bootstrap, r1))
    gamma = gamma_moment_1_estimator(k0_opti, r1)
    list_gamma.append(gamma)
plt.hist(list_gamma)
plt.title(f"estimation of gamma is {np.mean(list_gamma)} +/- {np.std(list_gamma)} vs true value : {1/inv_gamma}")

In [64]:
k0_opti = int(k0_opti)
gamma_moment_1_estimator(k0_opti, r1)

0.34148653081941965

In [ ]:
141